dcd_ch5_hw

# Chapter 5 Homework

In [25]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
pwd

'/content'

In [2]:
%cd /content/drive/MyDrive/GitHub

/content/drive/MyDrive/GitHub


In [3]:
%cd 'soil-physics-theory'

/content/drive/MyDrive/GitHub/soil-physics-theory


In [4]:
%cd 'content/chapter-5'

/content/drive/MyDrive/GitHub/soil-physics-theory/content/chapter-5


In [ ]:
from PSP_Marquardt import *

In [ ]:
%cd 'PSP_travelTimeAnalysis_NEW'

/content/drive/MyDrive/GitHub/soil-physics-theory/content/chapter-5/PSP_travelTimeAnalysis_NEW


In [7]:
from math import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
"""
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'
from tkinter import *
from google.colab.patches import cv2_imshow
from google.colab import output
from PIL import Image
"""

In [ ]:
cd ..

/content/drive/MyDrive/GitHub/soil-physics-theory/content/chapter-5


In [ ]:
c = 299792458
NODATA = -9999
MAXDELTAINDEX = 8
SX = 0
DX = 1

class CLine:
    a = NODATA
    b = NODATA

class CPoint:
    x = NODATA
    y = NODATA

line1 = CLine()
line2 = CLine()
lastFlatLine = CLine()

p0 = CPoint()
p1 = CPoint()
p2 = CPoint()
p3 = CPoint()

timeVector = []
reflecCoeff = []
dy =[]
dy2 =[]

deltaSpace = 0
deltaTime = 0

## Problem 1
**Use the program PSP_travelTimeAnalysis to compute the water content
of the soil sample dataTDRSoil.dat. Change the values of bulk density, temperature
and solid-phase dielectric permittivity to investigate the effects on the
computed water content by using the dielectric mixing model.**

In [ ]:
rawdata = readDataFile('dataTDRSoil.dat',0,'\t',0)

wv_avg = rawdata[0][0] #wave average
Vp = rawdata[0][1] #wave velocity
nrPoints = int(rawdata[0][2][0]) #numberof output points
cbl = rawdata[0][3] #cable length
windowWidth = rawdata[0][4] #window length
probeHandle = rawdata[0][5] #probe length
proff = rawdata[0][6] #probe offset
mult = rawdata[0][7] #multiplier
y = rawdata[0][8:]

FileNotFoundError: [Errno 2] No such file or directory: 'dataTDRSoil.dat'

In [ ]:
bulkDensity = 1350
first = 0
last = len(y)
index1 = indexOfMaxVector(y, first, last)
index2 = indexOfMinVector(y, first, last)
av = avg(y, index1, index2)
noV = normalizeVector(y)
bulkPermittivity =getBulkPermittivity(cbl+probeHandle, y, Vp)
theta = getWaterContentMalicki(bulkPermittivity[2], bulkDensity)
print(theta)

NameError: name 'y' is not defined

## Problem 2
**If the temperature of the soil liquid phase increases, does the bulk dielectric permittivity
increase or decrease? What effect would this change have on soil wate
content computation?**

In [ ]:
temps = range(0, 91, 30)

for i in temps:
  print("temperature = ",i)
  print("permiativity= ", getLiquidPermittivity(i))


temperature =  0
permiativity=  87.53086650000002
temperature =  30
permiativity=  76.7418267
temperature =  60
permiativity=  65.9527869
temperature =  90
permiativity=  55.16374710000001


As temperature increases the dielectric permittivity decreases.This is because, as the particle interactions increase the ability to resist electrical discharge increases.

## Problem 3
**Compute the matric water potential for water in a saturated pore 10 μm in diameter,
at 20 ◦C. The surface tension and liquid density at this temperature are
72.75 × 10–3 Nm–1 and 1027.8 kgm–3, respectively.**

In [ ]:
diam = 10
temp = 20
tension = 72.75 * 10**-3
rho_l = 1027.8
angle = 0
rad_meter = diam /2000000

psi_m = (2*tension*cos(angle))/(rho_l*rad_meter)
print("the matric potential, at a contact angle of 0 degrees, is", round(psi_m,2),'kPa')

the matric potential, at a contact angle of 0 degrees, is 28.31 kPa


## Problem 4
**By using the integration presented in the program
PSP_columnWaterContent.py, compute the total amount of soil water
in a clay soil for a profile that is 1m deep. Use the parameters presented in
Table 5.4**

In [ ]:
"""
#PSP_waterRetentionFitting
from __future__ import print_function, division
try: input = raw_input
except: pass

import numpy as np
import matplotlib.pyplot as plt
from PSP_readDataFile import readDataFile
from PSP_Marquardt import *

def main():
    # read experimental values
    myOutput, isFileOk = readDataFile(str(input('file name:')), 1, '\t', False)
    if (not isFileOk):
        print('Wrong file: error reading row nr.', myOutput)
        return(False)
    waterPotential = myOutput[:,0]
    waterContent = myOutput[:,1]

    # select water retention curve
    print (CAMPBELL,' Campbell')
    print (VAN_GENUCHTEN,' van Genuchten')
    print (RESTRICTED_VG,' van Genuchten with m = 1-1/n restriction')
    print (IPPISCH_VG,' Ippisch-van Genuchten')
    print (CAMPBELL_IPPISCH_VG,' Campbell-Ippisch-van Genuchten')

    waterRetentionCurve = 0
    while (waterRetentionCurve < CAMPBELL) or (waterRetentionCurve > CAMPBELL_IPPISCH_VG):
        waterRetentionCurve = float(input("Choose model type: "))
        if (waterRetentionCurve < CAMPBELL) or (waterRetentionCurve > CAMPBELL_IPPISCH_VG):
            print('wrong choice.')

    # initialize parameters
    thetaS = max(waterContent)
    #thetaR = min(waterContent)
    thetaR = 0.08
    air_entry = 1.0
    Campbell_b = 4.0
    VG_alpha = 1/air_entry
    VG_n = 1.5
    VG_m = 1. - 1./VG_n

    if (waterRetentionCurve == CAMPBELL):
        b0 = np.array([thetaS, air_entry, Campbell_b], float)
        bmin = np.array([thetaS, 0.1, 0.1], float)
        bmax = np.array([thetaS*1.1, 20., 10.], float)
    elif (waterRetentionCurve == VAN_GENUCHTEN):
        b0 = np.array([thetaS, thetaR, VG_alpha, VG_n, VG_m], float)
        bmin = np.array([thetaS, 0.0, 0.01, 0.01, 0.01], float)
        bmax = np.array([1.0, thetaR, 10., 10., 1.], float)
    elif (waterRetentionCurve == RESTRICTED_VG):
        b0 = np.array([thetaS, thetaR, VG_alpha, VG_n], float)
        bmin = np.array([thetaS, 0.0, 0.01, 1.], float)
        bmax = np.array([1, thetaR, 10., 10.], float)
    elif (waterRetentionCurve == IPPISCH_VG):
        b0 = np.array([thetaS, thetaR, air_entry, VG_alpha, VG_n], float)
        bmin = np.array([thetaS, 0.0, 0.1, 0.01, 1.], float)
        bmax = np.array([1, thetaR, 10., 10., 10.], float)
    elif (waterRetentionCurve == CAMPBELL_IPPISCH_VG):
        b0 = np.array([thetaS, thetaR, air_entry, VG_alpha, VG_n], float)
        bmin = np.array([thetaS, 0.0, 0.1, 0.01, 1.], float)
        bmax = np.array([1, thetaR, 10., 10., 10.], float)

    else:
        print ('wrong choice.')
        return(False)

    print ("\nFitting")
    b = Marquardt(waterRetentionCurve, b0, bmin, bmax, waterPotential, waterContent)

    print ("\nthetaS = ", b[0])
    if (waterRetentionCurve == CAMPBELL):
        print ("AirEntry = ", b[1])
        print ("b = ", b[2])
    elif (waterRetentionCurve == VAN_GENUCHTEN):
        print ("thetaR = ", b[1])
        print ("alpha = ", b[2])
        print ("n = ", b[3])
        print ("m = ", b[4])
    elif (waterRetentionCurve == RESTRICTED_VG):
        print ("thetaR = ", b[1])
        print ("alpha = ", b[2])
        print ("n = ", b[3])
    elif (waterRetentionCurve == IPPISCH_VG):
        print ("thetaR = ", b[1])
        print ("AirEntry = ", b[2])
        print ("alpha = ", b[3])
        print ("n = ", b[4])
    elif (waterRetentionCurve == CAMPBELL_IPPISCH_VG):
        print ("thetaR = ", b[1])
        print ("AirEntry = ", b[2])
        print ("alpha = ", b[3])
        print ("n = ", b[4])

    myWP = np.logspace(-5, 8, 500)
    myWC = estimate(waterRetentionCurve, b, myWP)

    plt.figure(figsize=(10,8))
    plt.plot(myWP, myWC,'k-')
    plt.plot(waterPotential, waterContent,'ko')

    plt.xscale('log')
    plt.xlabel('Water Potential [J kg$^{-1}$]',fontsize=20,labelpad=6)
    plt.xticks(size='16')
    plt.ylabel('Water Content [m$^{3}$ m$^{-3}$]',fontsize=20,labelpad=6)
    plt.tick_params(axis='both', which='major', labelsize=20,pad=8)
    plt.tick_params(axis='both', which='minor', labelsize=20,pad=8)
    #plt.savefig('waterRetention.eps')
    plt.show()

#main()
    """

## Problem 5
**By using the two files sand.txt and silt_loam.text, fit Campbell’s equation,
van Genuchten’s equation with no parameter restriction and van Genucthen’s
equation with the restriction m = 1 – 1/n. Print the parameters and discuss the differences
obtained by using the different models. Also discuss the differences in the
estimated parameters for the two soils.**

In [6]:
from main import main

In [ ]:
file_str = 'clay.txt'
model_int = 1
main(file_str, model_int)
file_str = 'clay.txt'
model_int = 2
main(file_str, model_int)
file_str = 'clay.txt'
model_int = 3
main(file_str, model_int)


In [ ]:
file_str = 'loam.txt'
model_int = 1
main(file_str, model_int)
file_str = 'loam.txt'
model_int = 2
main(file_str, model_int)
file_str = 'loam.txt'
model_int = 3
main(file_str, model_int)

## Problem 6
**Compute the water content at field capacity (assume ψFC = –33 J kg–1), and
permanent wilting point (assume ψPWP = –1500 J kg–1) using van Genuchten’s
equation for a soil having the following parameters: α = 0.15, n = 1.35, θr = 0.01
and θs = 0.46. The units of water potential are J kg–1. Plant available water is assumed
to be the difference between these values. Compute the plant available water
of the soil.**

In [ ]:
pot = [-33,-1500]


## Problem 7
**Derive the capacity for Campbell’s and van Genuchten’s equations by computing
the derivatives of the two functions with respect t psi**
.